This notebook uses an sgcn module in the bispy package to search the National List SGCN API. It returns and caches the summarized National List records, which include the list of states that have the species in their conservation planning process.

In [4]:
#Import needed packages
import bispy
import json
from IPython.display import display
from joblib import Parallel, delayed

sgcn = bispy.sgcn.Search()
bis_utils = bispy.bis.Utils()

In [5]:
# Open up cache updated ITIS list with valid/accepted species names from ITIS Exploration.ipynb
with open("cache/valid_itis.json", "r") as f:
    valid_itis = json.loads(f.read())

In [7]:
# Use joblib to run multiple requests for TESS documents in parallel via the valid/accepted ITIS scientific name
sgcn_result = Parallel(n_jobs=8)(delayed(sgcn.search)(name) for name in [r["scientific_name"] for r in valid_itis]) 
len(sgcn_result)

116

In [15]:
sgcn_result

[{'processing_metadata': {'status': 'failure',
   'date_processed': '2019-08-07T19:32:22.127460',
   'status_message': 'Not Matched',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Festuca idahoensis'},
  'parameters': {'Scientific Name': 'Festuca idahoensis',
   'Name Source': None}},
 {'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-07T19:32:22.127460',
   'status_message': 'Name Match',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Ursus americanus'},
  'parameters': {'Scientific Name': 'Ursus americanus', 'Name Source': None},
  'sgcn_species': {'statelist_2005': 'Alabama,Connecticut,Kansas,Kentucky,Massachusetts,Mississippi,Missouri,Ohio,Rhode Island,South Carolina,Texas,Vermont',
   'statelist_2015': 'Alabama,Kentucky,Massachusetts,Mississippi,Missouri,Ohio,Rhode Island,South Carolina,Texas',
   'scientificname': 'Ursus americanus',
   'commonname': 'American Black Bear',
 

In [8]:
# Filter to give just cases where SGCN species names matched with valid/accepted ITIS species names
sgcn_success=[i for i in sgcn_result if i['processing_metadata']['status'] == 'success']
len(sgcn_success)

57

In [9]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/sgcn.json", sgcn_success))

{'Doc Cache File': 'cache/sgcn.json',
 'Number of Documents in Cache': 57,
 'Document Number 6': {'processing_metadata': {'status': 'success',
   'date_processed': '2019-08-07T19:32:22.127460',
   'status_message': 'Name Match',
   'api': 'https://api.sciencebase.gov/bis-api/api/v1/swap/nationallist?scientificname=Centrocercus urophasianus'},
  'parameters': {'Scientific Name': 'Centrocercus urophasianus',
   'Name Source': None},
  'sgcn_species': {'statelist_2005': 'California,Colorado,Idaho,Montana,Nebraska,Nevada,North Dakota,Oregon,South Dakota,Utah,Washington,Wyoming',
   'statelist_2015': 'California,Colorado,Idaho,Montana,Nevada,North Dakota,Oregon,South Dakota,Utah,Washington,Wyoming',
   'scientificname': 'Centrocercus urophasianus',
   'commonname': 'Greater Sage Grouse',
   'taxonomicgroup': 'Birds',
   'taxonomicrank': 'Species',
   'matchmethod': 'Exact Match',
   'acceptedauthorityurl': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=17585

Here we look to see which WLCI species show up on the Wyoming SGCN list

In [10]:
sgcn_wy_names = []
for spp in [r["sgcn_species"] for r in sgcn_success if r["processing_metadata"]["status"] == "success"]:
    if "Wyoming" in spp["statelist_2005"] or "Wyoming" in spp["statelist_2015"]:
        print(spp["commonname"], '======', spp["scientificname"])
        sgcn_wy_names.append(spp["scientificname"])

Bell's Sparrow ====== Artemisiospiza belli
Uinta Ground Squirrel ====== Urocitellus armatus
Greater Sage Grouse ====== Centrocercus urophasianus
Brewer's Sparrow ====== Spizella breweri
cutthroat trout ====== Oncorhynchus clarkii
Ferruginous Hawk ====== Buteo regalis
mottled sculpin ====== Cottus bairdii
Bald Eagle ====== Haliaeetus leucocephalus
Sage Thrasher ====== Oreoscoptes montanus
mountain sucker ====== Catostomus platyrhynchus
moose ====== Alces alces
Colorado River cutthroat trout ====== Oncorhynchus clarkii pleuriticus
bighorn sheep ====== Ovis canadensis
Pygmy Rabbit ====== Brachylagus idahoensis
Brown Bear ====== Ursus arctos


Check to see if any invalid/not accepted ITIS specie names matched with SGCN records

In [11]:
# Open up the cached updated ITIS list with invalid/not accepted species names from ITIS Exploration.ipynb
with open("cache/invalid_itis.json", "r") as f:
    invalid_itis = json.loads(f.read())

In [12]:
# Use joblib to run multiple requests for SGCN documents in parallel via the invalid/not accepted ITIS scientific name
invalid_result = Parallel(n_jobs=8)(delayed(sgcn.search)(name) for name in [r["scientific_name"] for r in invalid_itis]) 

In [13]:
# Filter to give just cases where SGCN species names matched with invalid/not accepted ITIS names
invalid_result=[i for i in invalid_result if i['processing_metadata']['status'] == 'success']

In [14]:
len(invalid_result)

0

No SGCN records macthed with the invalid/not accepted ITIS specie names